In [1]:
# Pobranie bibliotek oraz modelu
!pip install transformers torch fastapi uvicorn nest-asyncio
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name="gpt2"


tokenizer = GPT2Tokenizer.from_pretrained(model_name, resume_download=True)#tokenizer zamienia słowo na ID tokenów
tokenizer.pad_token=tokenizer.eos_token #zamienia token dopełnienia PAD na token końca sekwencji EOS |GPT2 nie posiada własnego tokena pad
model = GPT2LMHeadModel.from_pretrained(model_name, resume_download=True)
model.eval() #tryb ewaluacji


if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model.to(device)


/home/wsl/.conda/envs/sage/lib/python3.9/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
from fastapi import FastAPI #framework do tworzenia REST api (m.in. obsługuje żądania HTTP) | FastAPI tworzy aplikacje serwerową
import nest_asyncio #pozwala uruchomić asynchroniczny kod w środowisku np. Jupyter
import uvicorn #Serwer ASGI (Asynchronous Server Gateway Interface), czyli serwer, który obsługuje FastAPI. To on nasłuchuje na porcie i odpowiada na żądania HTTP.
import asyncio

nest_asyncio.apply()  # potrzebne w Jupyterze

app = FastAPI()

@app.get("/")
def root():
    return {"status": "API GPT-2 działa", "endpoint": "/generate"}

@app.get("/info")
def info():
    return {
        "model": "gpt2",
        "vocab_size": model.config.vocab_size,
        "embedding_dim": model.config.n_embd
    }

@app.get("/generate")
def generate_text(prompt: str, max_length: int = 100):

    inputs = tokenizer(
        prompt,#tekst od użytkownika
        return_tensors="pt",#zwróć wynik jako tensor PyTorch
        padding=True #dopasowuje długość sekwencji dodając puste tokeny
    )

    input_ids = inputs["input_ids"].to(device)#id tokenów które trafiają do modelu
    print(f"Id tokenów w macierzy embedding:{input_ids}")
    attention_mask = inputs["attention_mask"].to(device)
    print(f"Attention mask:{attention_mask}\nPokazuje dla których tokenów liczyć prawdopodobieństwo a dla których nie")

    outputs = model.generate(
        input_ids=input_ids,#model bierze tokeny
        attention_mask=attention_mask,#przewiduje kolejne tokeny
        max_length=max_length,
        do_sample=True,#losuje kolejne tokeny if true, if False : zawsze najwieksze prawdopodobienstwo
        pad_token_id=tokenizer.eos_token_id #informuje model czym jest padding (czy dany token ignorować czy nie)
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)#zamiana wektorów na tekst
    return {"generated_text": text}


config=uvicorn.Config(app=app,host="127.0.0.1",port=5000,log_level="info")
server=uvicorn.Server(config=config)

asyncio.get_event_loop().run_until_complete(server.serve())
#aby uruchomić wpisz http://127.0.0.1:5000/generate?prompt=hello

INFO:     Started server process [2901]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


Id tokenów w macierzy embedding:tensor([[10919,   318,   262,  4252]])
Attention mask:tensor([[1, 1, 1, 1]])
Pokazuje dla których tokenów liczyć prawdopodobieństwo a dla których nie
INFO:     127.0.0.1:63039 - "GET /generate?prompt=what%20is%20the%20sun HTTP/1.1" 200 OK


**Tokenizacja** to proces zamiany tekstu na sekwencję identyfikatorów tokenów (input_ids), które indeksują wektory embeddingów, oraz maski uwagi (attention_mask), która informuje model, które tokeny powinny być brane pod uwagę podczas obliczeń.

**GPT-2 używa Byte Pair Encoding (BPE)** → dzieli tekst na subtokeny, a nie słowa.

In [ ]:
words = ["sztuczna", "inteligencja", "przyszłość"]

# Tokeny GPT-2
tokens = [tokenizer.encode(w, add_special_tokens=False) for w in words]
print("Tokeny:", tokens)

# Embeddingi dla tokenów
embeddings = model.transformer.wte.weight  # wte = word token embeddings
for i, w_tokens in enumerate(tokens):
    for t in w_tokens:
        print(f"Słowo: {words[i]}, Token: {t}, Embedding shape: {embeddings[t].shape}")


In [ ]:
prompt = "Sztuczna inteligencja"
inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

# Forward pass, bez generacji
with torch.no_grad():
    outputs = model(inputs)
    logits = outputs.logits  # shape: [1, seq_len, vocab_size]

# Softmax dla ostatniego tokena
import torch.nn.functional as F
last_token_logits = logits[0, -1, :]
probs = F.softmax(last_token_logits, dim=-1)

# 100 najbardziej prawdopodobnych tokenów
top_k = torch.topk(probs, 100)
for token_id, prob in zip(top_k.indices.tolist(), top_k.values.tolist()):
    print(tokenizer.decode([token_id]), prob)


In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(100), top_k.values.cpu().numpy())
plt.title("Top 100 prawdopodobnych tokenów")
plt.show()
